In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd
from tabulate import tabulate

# Trainer
from src.trainer.trainer import Trainer

# Models
from src.models.mlp import MLP
from src.models.p4_allcnn import P4AllCNNC
from src.models.fA_p4_allcnn import fA_P4AllCNNC
from src.models.p4m_allcnn import  P4MAllCNNC
from src.models.fA_p4m_allcnn import fA_P4MAllCNNC
from src.models.p4m_resnet import  P4MResNet
from src.models.fA_p4_resnet import fA_P4MResNet


## CIFAR10 dataset

In [2]:
root_path = "../data"
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

full_train = datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform_train)

train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])
test_set = datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform_test)


batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [3]:
img, label = full_train[0]
print(img.shape)

torch.Size([3, 32, 32])


## Experiment

In [4]:
# ----- Helper Functions -----
def init_model(name):
    if name == "mlp":
        return MLP(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size)
    elif name == "p4_allcnn":
        return P4AllCNNC()
    elif name == "fA_p4_allcnn":
        return fA_P4AllCNNC()
    elif name == "p4m_allcnn":
        return P4MAllCNNC()
    elif name == "fA_p4m_allcnn":
        return fA_P4MAllCNNC()
    elif name == "p4m_resnet":
        return P4MResNet()
    elif name == "fA_p4_resnet":
        return fA_P4MResNet()
    else:
        raise ValueError(f"Unknown model name: {name}")

def init_optimizer(model):
    return optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

def init_scheduler(optimizer):
    return optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 250, 300], gamma=0.1)

In [5]:
# ----- Configuration -----
num_epochs = 1
num_iterations = 1
log_dir = "../logs"

model_names = [
    "mlp",
    "p4_allcnn",
    "fA_p4_allcnn",
    "p4m_allcnn",
    "fA_p4m_allcnn",
    "p4m_resnet",
    "fA_p4_resnet"
]

accuracies = {name: [] for name in model_names}

# ----- Model-specific settings -----
input_size = 3 * 32 * 32
output_size = 10
hidden_sizes = [128, 64]

# ----- Main Training Loop -----
for it in range(num_iterations):
    print(f"Iteration {it + 1}/{num_iterations}")

    models = {name: init_model(name) for name in model_names}
    criterions = {name: nn.CrossEntropyLoss() for name in model_names}
    optimizers = {name: init_optimizer(models[name]) for name in model_names}
    schedulers = {name: init_scheduler(optimizers[name]) for name in model_names}

    trainer = Trainer(
        models=models,
        optimizers=list(optimizers.values()),
        criterions=list(criterions.values()),
        schedulers=list(schedulers.values()),
        log_dir=log_dir
    )

    print("Training models...")
    trainer.train(
        num_epochs=num_epochs,
        train_loader=train_loader,
        val_loader=val_loader,
    )

    print("Testing models...")
    test_accuracies = trainer.evaluate(test_loader=test_loader)

    for name, acc in test_accuracies.items():
        accuracies[name].append(acc)

Iteration 1/1
Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.02s/it, Train Acc=0.2251, Val Acc=0.2414, Train Loss=2.0667, Val Loss=2.0243]


Training model : p4_allcnn


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:52<00:00, 52.83s/it, Train Acc=0.1816, Val Acc=0.1816, Train Loss=2.2291, Val Loss=2.2660]


Training model : fA_p4_allcnn


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 1/1 [01:22<00:00, 82.70s/it, Train Acc=0.2414, Val Acc=0.1788, Train Loss=2.0412, Val Loss=2.8473]


Training model : p4m_allcnn


Training Epochs:   0%|                                                                                                                                                  | 0/1 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 4)

In [ ]:
# ----- Final Statistics -----
final_stats = {
    name: {
        "% Test error": (1 - float(np.mean(vals))) * 100,
        "% std": float(np.std(vals)) * 100,
        "Num Parameters": sum(p.numel() for p in init_model(name).parameters())
    }
    for name, vals in accuracies.items()
}

## Table generation

In [ ]:
df = pd.DataFrame.from_dict(final_stats, orient='index')
df = df.round(2)
print("📊 Model Accuracy Summary in CIFAR10\n")
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))